IV. napari
==========

_Image Analysis with Python and Napari, Bioinformatics for Ukraine course, 6-24 October 2025, Kyiv, Ukraine._

_© Borys Olifirov, 2025_

__План:__
- Огліяд napari
- Розширення функціоналу napari за допомогою віджетів
- Оформлення набору функцій як плгіну napari

---

In [ ]:
import os
import pathlib
import datetime

import numpy as np
import skimage  # scikit-image
import matplotlib.pyplot as plt

from napari import Viewer                # головний клас переглядача napari
from napari.layers import Image, Labels  # різні типи шарів napari
from napari.utils.notifications import show_info  # для виведення повідомлень у napari
from magicgui import magic_factory       # для створення віджетів napari

# napari viewer та різні типи шарів
---

### Завантаження зображень як шару Image

##### Демонстраційне 2D-зображення

In [ ]:
neuron_img_2D = skimage.io.imread('demo_data/2D_grey_matter_neurofilaments.tif')
print(neuron_img_2D.shape)

plt.imshow(neuron_img_2D, cmap='gray')

Передамо зображення як шар Image до переглядача napari та запустимо переглядач

In [ ]:
# створюємо об'єкт переглядача napari
viewer = Viewer()

# додаємо шар Image з нашим зображенням, вказуємо ім'я шару та кольорову мапу
viewer.add_image(neuron_img_2D, name='neuron_img_2D', colormap='gray')
viewer.show()

##### Демонстраційне 4D-зображення

In [ ]:
nerve_img_4D = skimage.io.imread('demo_data/4D_nerve_spectral_z-stack.tif')
print(nerve_img_4D.shape)


In [ ]:
viewer = Viewer()
viewer.add_image(nerve_img_4D, name='nerve_img_4D', colormap='gray')
viewer.show()

Розділемо спектральні канали та відобразимо їх як окремі шари Image

In [ ]:
nerve_img_ch0 = nerve_img_4D[:,0]
nerve_img_ch1 = nerve_img_4D[:,1]

print(nerve_img_ch0.shape, nerve_img_ch1.shape)

In [ ]:
viewer = Viewer()
viewer.add_image(nerve_img_ch0, name='nerve_img_ch0', colormap='gray')
viewer.add_image(nerve_img_ch1, name='nerve_img_ch1', colormap='gray')
viewer.show()

### Завантаження маски чи ярликів як шару Labels

##### Демонстраційна 2D-маска

In [ ]:
neuron_mask_2D = skimage.io.imread('course_data/neuron_mask.tif')
print(neuron_mask_2D.shape)

plt.imshow(neuron_mask_2D, cmap='gray')

In [ ]:
viewer = Viewer()
viewer.add_labels(neuron_mask_2D, name='neuron_mask_2D', opacity=0.75)
viewer.show()

##### Демонстраційна 3D-маска

Завантажимо z-стек клітини HEK 293

In [ ]:
hek_img_3d = skimage.io.imread('demo_data/3D_HEK_z-stack.tif')
print(hek_img_3d.shape)

plt.imshow(hek_img_3d[5], cmap='magma')

Завантажимо 3D-маску клітини HEK 293

In [ ]:
hek_mask_3d = skimage.io.imread('course_data/3D_hek_mask.tif')
print(hek_mask_3d.shape)

plt.imshow(hek_mask_3d[5], cmap='grey')

Додамо шар Image та шар Labels до переглядача napari

In [ ]:
viewer = Viewer()
viewer.add_image(hek_img_3d, name='HEK 293', colormap='grey')
viewer.add_labels(hek_mask_3d, name='HEK 293 Mask', opacity=0.5)
viewer.show()

### Інші типи шарів napari
Ці типи шарів можуть бути створені вручну і використані для розмітки даних:
- __Points__ - для відображення набору точкових координат, задається набором координат.
- __Shapes__ - для відображення фігур (ліній, прямокутників, еліпсів, багатокутників).

Ці типи шарів можуть бути створені виключно програмно:
- __Surface__ - для відображення 3D поверхонь, задається набором координат вершин та граней.
- __Traces__ - для відображення змін координат у часі, задається набором координат і часових точок.
- __Vectors__ - для відображення напрямків, задається набором координат початків та напрямків.

Детальний опис можна знайти у [документації napari](https://napari.org/stable/howtos/layers/index.html)

# Віджети napari
---

### Пригадаємо функцію для побудови простої маски

In [ ]:
# Можете позичити цю функцію для власного модуля
def simple_masking(image: np.ndarray, median_filter:int=0,
                   closing:int=3, opening:int=3):
    # попередня обробка зображення медіанним фільтром
    preprocessed_image = skimage.filters.median(image,
                                                footprint=skimage.morphology.disk(median_filter))
    
    # побудова маски за допомогою порогу Отсу
    otsu_mask = preprocessed_image > skimage.filters.threshold_otsu(preprocessed_image)
    
    # закриттям маски позбавляємось дрібних прогалин
    pre_filtered_mask = skimage.morphology.closing(otsu_mask,
                                                   footprint=skimage.morphology.disk(closing))

    # відкриттям маски видаляємо дрібні артефакти поза клітиною
    filtered_mask = skimage.morphology.opening(pre_filtered_mask,
                                               footprint=skimage.morphology.disk(opening))
    
    return filtered_mask

In [ ]:
neuron_mask = simple_masking(neuron_img_2D, median_filter=3, closing=5, opening=5)

plt.imshow(neuron_mask, cmap='gray')

### Перетворимо функцію на віджет napari

Модифікована функція з використанням декоратора `@magic_factory` для створення віджету

In [ ]:
@magic_factory(call_button='Create Mask')
def simple_masking_widget(viewer: Viewer, image: Image,
                          median_filter:int=0, closing:int=3, opening:int=3):
    image_data = image.data  # збережемо дані в тимчасову змінну

    # виведемо ім'я шару та розмірність зображення
    show_info(f'Image name: {image.name}')
    show_info(f'Image shape: {image_data.shape}, dtype: {image_data.dtype}')

    # попередня обробка зображення медіанним фільтром
    preprocessed_image = skimage.filters.median(image_data,
                                                footprint=skimage.morphology.disk(median_filter))
    # побудова маски за допомогою порогу Отсу
    otsu_mask = preprocessed_image > skimage.filters.threshold_otsu(preprocessed_image)
    # закриттям маски позбавляємось дрібних прогалин
    pre_filtered_mask = skimage.morphology.closing(otsu_mask,
                                                   footprint=skimage.morphology.disk(closing))
    # відкриттям маски видаляємо дрібні артефакти поза клітиною
    filtered_mask = skimage.morphology.opening(pre_filtered_mask,
                                               footprint=skimage.morphology.disk(opening))

    # додаємо шар Labels з отриманою маскою    
    viewer.add_labels(filtered_mask, name=f'{image.name}_simple_mask', opacity=0.5)

Створимо екземпляр віджету та додамо його до переглядача napari

In [ ]:
smw = simple_masking_widget()

viewer = Viewer()
viewer.add_image(neuron_img_2D, name='neuron_img_2D', colormap='gray')
viewer.window.add_dock_widget(smw,
                              name = 'Simple Masking',
                              area='right')
viewer.show()

### Додамо інтерактивні елементи налаштувань

Приклад інтерактивних елементів для різних типів параметрів

In [ ]:
@magic_factory(call_button="Press me",
               slider_float={"widget_type": "FloatSlider", 'min': -5, 'max': 5},
               slider_int={"widget_type": "IntSlider", 'min': -5, 'max': 5},
               dropdown={"choices": ['first', 'second', 'third']},)
def widget_demo(viewer: Viewer,
                maybe: bool,
                some_int: int,
                spin_float:float=3.14159,
                slider_float:float=2.71828,
                slider_int:int=3,
                string:str="Text goes here",
                dropdown:str='first',
                date=datetime.datetime.now(),
                filename=pathlib.Path('/some/path.ext')):
    ''' Widget fields example
    
    '''
    show_info(f'Buttom pressed')

Приклад використання віджета просто як обгортки для звичайної функції

In [ ]:
@magic_factory(call_button="Build Mask",
               med={"widget_type": "IntSlider", 'min': 0, 'max': 10},
               clsng={"widget_type": "IntSlider", 'min': 0, 'max': 10},
               opnng={"widget_type": "IntSlider", 'min': 0, 'max': 10})
def simple_masking_envelope(viewer: Viewer, image: Image,
                            med:int=0, clsng:int=3, opnng:int=3):
    image_data = image.data  # збережемо дані в тимчасову змінну

    # виведемо ім'я шару та розмірність зображення
    show_info(f'Image name: {image.name}')
    show_info(f'Image shape: {image_data.shape}, dtype: {image_data.dtype}')

    # викликаємо нашу функцію обробки зображення
    output_mask = simple_masking(image_data,
                                 median_filter=med,
                                 closing=clsng,
                                 opening=opnng)

    # додаємо шар Labels з отриманою маскою
    viewer.add_labels(output_mask, name=f'{image.name}_simple_mask', opacity=0.5)

In [ ]:
wd = widget_demo()
sme = simple_masking_envelope()

viewer = Viewer()
viewer.add_image(neuron_img_2D, name='neuron_img_2D', colormap='gray')

viewer.window.add_dock_widget(wd,
                              name = 'Demo',
                              area='right')
viewer.window.add_dock_widget(sme,
                              name = 'Nice Simple Masking',
                              area='right')

viewer.show()

# Плігни napari
---

Плагін napari - це паке Python, що містить додатковий службовий файл маніфісту napari `napari.yaml`, який описує призначення окремих функцій в межах роботи napari.

Плагін може бути встановлений як звичайний пакет Python за допомогою pip, conda. Вже інсуючі плагіни можна знайти в [каталозі плагінів napari](https://napari-hub.org/).

Додаткову інформацію про створення плагінів можна подивитись в [офіціфній документації napari](https://napari.org/dev/plugins/building_a_plugin/first_plugin.html).

Доступний шаблон плагіну napari можна знайти в `templates/plugin-template`.

__Структура плагіну napari__

```
└── plugin-template/          # директорія плагіну
    ├── src/                  # загальна директорія із вихідним кодом
    │   └── plugin_template/      # директорія із модулями плагіну
    │       ├── __init__.py       # службовий файл директорії
    │       ├── napari.yaml       # маніфест napari
    │       └── _widget.py        # код віджетів плагіну
    │
    ├─── pyproject.toml       # конфігураційний файл
    ├─── README.md            # файл опису
    ├─── LICENSE              # ліцензія
    └─── .gitignore           # службовий файл git
```

# Фінальний проєкт
---

Користуючись напрацюванями з попередніх зустрічей створіть плагін napari, що реалізує основні кроки обробки зображень:
- Попередня обробка (фільтрація, згладжування)
- Сегментація (поріг, морфологія)
- Оцінка параметрів об'єктів та збереження результатів